# Part 3: Quantification

In this notebook, we will look at ways to quantitatively characterize objects as they appear in microscopy images. Many different types of measurements can be extracted from images. The ones that are most relevant is ultimately dependent on the underlying biological question. Here, we will look at commonly-used (non-machine-learning) strategies to quantify information that pertains to image intensity and object shape.

In [ ]:
import os
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 200

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import skimage as sk
import skimage.measure as skm

In this part, we will use pandas to handle numerical data (https://pandas.pydata.org/). Feel free to consult the extensive documentation available on their website if you want to know more about this library!

In [ ]:
import pandas as pd

## 1. Data loading

We will again work with images from the BBBC datasets introduced in notebook 1 - Handling Image Data. 

**1.1** Using what you learned in notebook 1 - Handling Image Data, load the following image files:
- data/Part 3/BBBC010/raw.tif
- data/Part 3/BBBC020/membranes.tif

*Do not forget to convert the images into floating-point arrays with img_as_float!*

In [ ]:
# BBBC010
##### Add you code here ##### 

# BBBC020
##### Add you code here ##### 

**1.2** To help us with the quantification, we also have instance segmentation masks provided for each image in a folder called "instances". Run the lines below to load these masks into an array.

In [ ]:
# BBBC010
bbbc010_instances = {}
idx = 0
for file in os.listdir('data/Part 3/BBBC010/instances'):
    if file.endswith(".png"):
        im = imageio.imread(os.path.join('data/Part 3/BBBC010/instances',file))
        bbbc010_instances[idx] = im
        idx+=1
                            
# BBBC020
bbbc020_instances = {}
idx = 0
for file in os.listdir('data/Part 3/BBBC020/instances'):
    if file.endswith(".TIF"):
        im = imageio.imread(os.path.join('data/Part 3/BBBC020/instances',file))
        bbbc020_instances[idx] = im
        idx+=1

## 2. Intensity-based measurements

The most direct measurements one can make are those based on pixel values. When dealing with raw (unprocessed) images, pixel values are related to the amount of light (photons) that reached the camera during the acquisition process - although the exact nature of this relationship may be quite complicated. 

**2.1** The BBBC020 image corresponds to the signal of a fluorescently-tagged cell-surface protein (CD11b/APC). Run the code below to extract some statistics about the intensity distributions for each objects in the image using the provided instance segmentation masks. Are the results surprising? 

In [ ]:
# Extract intensity features
bbbc020_intensity_feats = []
for idx in bbbc020_instances.keys():
    mask = bbbc020_instances[idx]
    
    intensities = bbbc020[mask>0] 
    median = np.median(intensities)
    std = np.std(intensities)
    
    bbbc020_intensity_feats.append([median, std])
    
bbbc020_intensity_feats = pd.DataFrame(bbbc020_intensity_feats, columns=['Median', 'Standard Deviation'])
bbbc020_intensity_feats.index.name = 'Instance ID'

display(bbbc020_intensity_feats)

In [ ]:
# Visualize the distribution of each feature as an histogram
fig, axes = plt.subplots(nrows=1, ncols=2)

axes[0].hist(bbbc020_intensity_feats['Median'].values) 
axes[0].set_title('Median')

axes[1].hist(bbbc020_intensity_feats['Standard Deviation'].values)  
axes[1].set_title('Standard Deviation')

plt.show()

**2.2** The BBBC010 image is a brightfield view of *C. elegans* worms treated with an antibiotic. The worms are also stained with a fluorescent protein that specifically targets dead animals. Run the code below to load the fluorescence microscopy readout, and adapt the code from 2.1 to study the population of worms in the BBBC010 image. Can you spot the dead worms?

In [ ]:
# Load fluorescence readout
bbbc010_fluo = imageio.imread('data/Part 3/BBBC010/fluorescence.tif')
bbbc010_fluo = sk.img_as_float(bbbc010_fluo)

In [ ]:
# Extract intensity features from fluorescence readout
##### Add you code here ##### 

In [ ]:
# Visualize the distribution of each feature as an histogram
##### Add you code here ##### 

## 3. Shape-based measurements

In addition to the intensity of pixels within them, the shape of objects in images can hold relevant information. Shape is generally challenging to quantify with a single number, but can be reasonably characterized with collections of descriptors as we shall see.

**3.1** The most classical and direct way of characterizing shape is to rely on collection of measurements referred to as *region properties*. Run the code below to extract a subset of scikit-image's own collection of region properties on the BBBC020 image. Their definition is provided at https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops . Do you understand which aspect of the shape they capture?

Below, you will also find a piece of code that colors instances in the image according to the value of a shape property of your choice. Play around with it to get an intuition of what these different metrics reveal.

In [ ]:
# Extract shape features
bbbc020_shape_feats = []
for idx in bbbc020_instances.keys():
    mask = bbbc020_instances[idx]
    
    all_props = skm.regionprops(mask)[0]
    bbbc020_shape_feats.append([all_props['area'], all_props['perimeter'], all_props['axis_major_length'],
                        all_props['axis_minor_length'], all_props['eccentricity'], all_props['solidity'],
                        all_props['extent']])

bbbc020_shape_feats = pd.DataFrame(bbbc020_shape_feats, columns=['Area', 'Perimeter', 'Major axis', 'Minor axis',
                                                 'Eccentricity', 'Solidity', 'Extent'])
bbbc020_shape_feats.index.name = 'Instance ID'

display(bbbc020_shape_feats)

In [ ]:
# Label objects according to a property of your choice
feature = 'Area' ##### Try to modify this #####

feature_mask = np.zeros(bbbc020.shape)
for idx in bbbc020_instances.keys():
    mask = bbbc020_instances[idx]
    
    feature_val = bbbc020_shape_feats.loc[idx, feature]
    feature_mask[np.where(mask)] = feature_val

plt.imshow(bbbc020, cmap='gray')
im = plt.imshow(feature_mask, cmap='inferno', alpha=.4)
plt.colorbar(im, fraction=0.05, pad=0.01)
plt.axis('off')
plt.title('Color-coded by '+feature, fontsize=9)
plt.show()

**3.2** As an alternative to individual independent measurements as exactred from region propertiers, one can summarize a shape by the coefficients of its decomposition into an orthonormal basis. This is the idea behind Fourier descriptors (https://en.wikipedia.org/wiki/Fourier_analysis), implemented in the pyefd library (https://github.com/hbldh/pyefd).

In [ ]:
import pyefd

Although the technical details of Fourier analysis is beyond the scope of this course, we can get a visual intuition of how that works. Run the code below to 1) decompose the shape of one of the objects in the BBBC020 image in the Fourier basis, 2) visualize its N-term approximation. By varying N, can you get a sense of how information is distributed across Fourier coefficients?

In [ ]:
N = 1 ##### Try to modify this #####

for idx in bbbc020_instances.keys():
    mask = bbbc020_instances[idx]
    
    # Retreive instance contour
    outline = skm.find_contours(mask,1)
    outline = outline[0]
    
    # Decompose into the Fourier basis
    (A0,C0) = pyefd.calculate_dc_coefficients(outline)
    fourier_coefficients = pyefd.elliptic_fourier_descriptors(outline, order = N)
    
    # Reconstruct
    n_term_approximation = pyefd.reconstruct_contour(fourier_coefficients, locus = (A0, C0), num_points = len(outline))
    
    # Visualize the result
    fig, axes = plt.subplots(nrows=1, ncols=2)
    
    axes[0].scatter(outline[:,1], outline[:,0], s = 5, c = 'red')
    axes[0].axis('equal')
    axes[0].axis('off')
    axes[0].set_title("Original shape")

    axes[1].scatter(n_term_approximation[:,1], n_term_approximation[:,0], s = 5, c = 'red')
    axes[1].axis('equal')
    axes[1].axis('off')
    axes[1].set_title(str(N)+"-term approximation")
    
    fig.tight_layout()
    plt.show()
    

**3.3** When *C. elegans* worms die, they rigidify into a straight rod. Even in the absence of the extra fluorescence microscopy readout, we could therefore still try to assess whether the *C. elegans* worms in the BBBC010 image are alive or not. Adapt 3.1 to use measurements from the list of region properties that you think are appropriate to capture the "straight" shape phenotype, and study the population of worms in the BBBC010 image using only the brightfield data. Do you manage to reach the same conclusion as in 2.2?

In [ ]:
# Extract shape features
##### Add you code here ##### 

In [ ]:
# Visualize the distribution of each feature as an histogram
##### Add you code here ##### 

## 4. Export

A collection of measurements describing an object as it appears on image data is referred to as a *feature vector*. Feature vectors are the basic ingredient of data analysis: they allow to use the visual information contained in images in statistical studies.

**4.1** Run the lines below to assemble all the intensity (2.2) and shape (3.3) measurements you have extracted on each object of the BBBC010 image into one big matrix.

In [ ]:
feature_matrix = pd.concat([bbbc010_intensity_feats, bbbc010_shape_feats], axis=1)

display(feature_matrix)

**4.2** Run the line below to save the *feature matrix* you compiled into a spreadsheet. This file now quantitatively summarizes the content of the fluorescence and brightfield image and can be used for futher analysis.

In [ ]:
feature_matrix.to_csv('data/Part 3/bbbc010_features.csv')